### Enigma Machine
Angie Hollingworth

This is by no means a useable encryption method, given that there are many floors not adressed such as string length and ASCII character values outside of the given range. (thse just default to a '?')

However, this was a nice challenge on creating an encryption method that can be decrypted and offers some level of complexity.

In [1]:
import math
import random

In [2]:
msg = "The first use of the term 'cryptograph' (as opposed to 'cryptogram') dates back to the 19th century-originating from 'The Gold-Bug,' a story by Edgar Allan Poe.[12][13][broken footnote] Until modern times, cryptography referred almost exclusively to 'encryption', which is the process of converting ordinary information (called plaintext) into an unintelligible form (called ciphertext).[14] Decryption is the reverse, in other words, moving from the unintelligible ciphertext back to plaintext. A cipher (or cypher) is a pair of algorithms that carry out the encryption and the reversing decryption. The detailed operation of a cipher is controlled both by the algorithm and, in each instance, by a 'key'. The key is a secret (ideally known only to the communicants), usually a string of characters (ideally short so it can be remembered by the user), which is needed to decrypt the ciphertext. In formal mathematical terms, a 'cryptosystem' is the ordered list of elements of finite possible plaintexts, finite possible cyphertexts, finite possible keys, and the encryption and decryption algorithms that correspond to each key. Keys are important both formally and in actual practice, as ciphers without variable keys can be trivially broken with only the knowledge of the cipher used and are therefore useless (or even counter-productive) for most purposes. Historically, ciphers were often used directly for encryption or decryption without additional procedures such as authentication or integrity checks. There are two main types of cryptosystems: symmetric and asymmetric. In symmetric systems, the only ones known until the 1970s, the same secret key encrypts and decrypts a message. Data manipulation in symmetric systems is significantly faster than in asymmetric systems. Asymmetric systems use a 'public key' to encrypt a message and a related 'private key' to decrypt it. The advantage of asymmetric systems is that the public key can be freely published, allowing parties to establish secure communication without having a shared secret key. In practice, asymmetric systems are used to first exchange a secret key, and then secure communication proceeds via a more efficient symmetric system using that key.[15] Examples of asymmetric systems include Diffie-Hellman key exchange, RSA (Rivest-Shamir-Adleman), ECC (Elliptic Curve Cryptography), and Post-quantum cryptography. Secure symmetric algorithms include the commonly used AES (Advanced Encryption Standard) which replaced the older DES (Data Encryption Standard).[16] Insecure symmetric algorithms include children's language tangling schemes such as Pig Latin or other cant, and all historical cryptographic schemes, however seriously intended, prior to the invention of the one-time pad early in the 20th century. In colloquial use, the term 'code' is often used to mean any method of encryption or concealment of meaning. However, in cryptography, code has a more specific meaning: the replacement of a unit of plaintext (i.e., a meaningful word or phrase) with a code word (for example, 'wallaby' replaces 'attack at dawn'). A cypher, in contrast, is a scheme for changing or substituting an element below such a level (a letter, a syllable, or a pair of letters, etc.) in order to produce a cyphertext. Cryptanalysis is the term used for the study of methods for obtaining the meaning of encrypted information without access to the key normally required to do so; i.e., it is the study of how to 'crack' encryption algorithms or their implementations. Some use the terms 'cryptography' and 'cryptology' interchangeably in English,[17] while others (including US military practice generally) use 'cryptography' to refer specifically to the use and practice of cryptographic techniques and 'cryptology' to refer to the combined study of cryptography and cryptanalysis.[18][19] English is more flexible than several other languages in which 'cryptology' (done by cryptologists) is always used in the second sense above. RFC 2828 advises that steganography is sometimes included in cryptology.[20] The study of characteristics of languages that have some application in cryptography or cryptology (e.g. frequency data, letter combinations, universal patterns, etc.) is called cryptolinguistics."

In [3]:
class Enigma():
    def __init__(self):
        '''Instead of switching characters, divide the entire set of values into a given subset of characters, 
            and swap their indexes for the plugs'''
        self.rotator_one = None
        self.rotator_two = None
        self.rotator_three = None
        self.rotator_one_shift = 1
        self.rotator_two_shift = 2
        self.rotator_three_shift = 3
        self.plugs = []
        self.ind = []
        self.size = 0
        self.ind_one = []
        self.ind_two = []
        self.ind_three = []
        self.rotate = 25
        
    def set_machine(self, rotators = False, shift = False, size = 0, plugs = False, rotate = False):
        if rotators == False:
            # Only ASCII characters from 32 - 125
            self.rotator_one = chr(random.randint(32,126))
            self.rotator_two = chr(random.randint(32,126))
            self.rotator_three = chr(random.randint(32,126))
        else:
            self.rotator_one = rotators[0]
            self.rotator_two = rotators[1]
            self.rotator_three = rotators[2]
        if shift == False:
            self.rotator_one_shift = random.randint(3,20)
            self.rotator_two_shift = random.randint(21,50)
            self.rotator_three_shift = random.randint(5,90)
        else:
            self.rotator_one_shift = shift[0]
            self.rotator_two_shift = shift[1]
            self.rotator_three_shift = shift[2]
        if size != 0:
            self.size = abs(size)
        else:
            self.size = random.randint(5,94//2) # Limit the number of possible divisions to be in a minimal set of pairs
        if plugs == False:
            ''' Randomise order of character division '''
            self.plugs = [i for i in range(0,self.size)]
            random.shuffle(self.plugs) 
        else:
            self.plugs = plugs
            self.size = len(plugs) #avoid misatch in length
        self.build_index()
        if rotate:
            self.rotate = abs(rotate)
        # Set rotators
        self.set_rotator_indexes(self.rotator_one, self.rotator_one_shift, self.ind_one, self.ind)
        self.set_rotator_indexes(self.rotator_two, self.rotator_two_shift, self.ind_two, self.ind)
        self.set_rotator_indexes(self.rotator_three, self.rotator_three_shift, self.ind_three, self.ind)

    
    def settings(self, p = False):
        if p:
            self.print_settings()        
        return [self.rotator_one, self.rotator_two, self.rotator_three, self.rotator_one_shift, self.rotator_two_shift, self.rotator_three_shift, self.size, self.plugs, self.rotate]
    
    def print_settings(self):
        print("\n*** SETTINGS FOR MACHINE ***")
        print(f'R1: {self.rotator_one}\nR2: {self.rotator_two}\nR3: {self.rotator_three}\n')
        print(f'R1 Shift: {self.rotator_one_shift}\nR2 Shift: {self.rotator_two_shift}\nR3 Shift: {self.rotator_three_shift}\n')
        print(f'Plug Size: {self.size}')
        print(f'Plug Order: {[i for i in self.plugs]}')
        
    def set_rotator_indexes(self, rotator, shift, set_list, index_list):
        a = index_list.index(ord(rotator))+ shift
        if a >= len(index_list):
            a = a- len(index_list)
        for i in range(a, len(index_list)):
            set_list.extend([index_list[i]])
        for i in range(0, a):
            set_list.extend([index_list[i]])

    def encrypt(self, text):
        rotate_length = len(self.ind)
        r2 = 1
        r3 = 1
        text = self.switch_ascii_value(text)
        text_o = []
        text_c = []
        for i, n in enumerate(text):
            # check for characters outside of range, replace with ?
            if n < 32 or n >124:
                n = ord('?')
            # get index in base
            bi = self.ind.index(n)
            # Add shift One to bi
            oi = bi+(i*self.rotator_one_shift)
            oi = oi % rotate_length
            # Get letter from one
            on = self.ind_one[oi]
            # get index from 2
            i2 = self.ind_two.index(on)
            # add two shift
            if i >0 and i % self.rotate == 0:
                i2+= self.rotator_two_shift*r2
                r2 +=1
                i2 = i2 % rotate_length
            # Add three shift
            i3 = i2
            if i >0 and i % (self.rotate*self.rotate) == 0:
                i2 += self.rotator_three_shift*r3
                r3+=1
            v = i3 % rotate_length
            #add to list
            v3 = self.ind_three[v]
            text_o.extend([v3])
        #convert list
        text_c = self.switch_ascii_value(text_o,'c')
        return ''.join(text_c)
            
    def decrypt(self, text):
        rotate_length = len(self.ind)
        r2 = 1
        r3 = 1
#         message_length = len(text)
        text = self.switch_ascii_value(text) #''' KEEP THIS TEXT SWITCH'''
        text_o = []
        text_c = []
        for i, n in enumerate(text):
            i3 = self.ind_three.index(n)
            i2 = i3
            # Deduct shift 3 from i2
            if i >0 and i % (self.rotate*self.rotate) == 0:
                i2 -= self.rotator_three_shift*r3
                r3+=1
                i2 = i2 % rotate_length
            i1 = i2
            if i >0 and i % self.rotate == 0:
                i1 -= self.rotator_two_shift*r2
                r2 +=1
                i1 = i1 % rotate_length
            # get leter from two with shifted index
            v2 = self.ind_two[i1]
            # get index from one with letter
            b1 = self.ind_one.index(v2)
            # shift index one
            bi = b1 - (i*self.rotator_one_shift)
            bi = bi%rotate_length            
            v = self.ind[bi]            
            #add to list
            text_o.extend([v])
            #convert list
        text_c = self.switch_ascii_value(text_o,'c')
        return ''.join(text_c)
    
    def switch_ascii_value(self,text, flag = 'o'):        
        text_string = []
        if flag == 'o':        
            for letter in text:
                text_string.extend([ord(letter)])
        elif flag == 'c':
            for letter in text:
                text_string.extend([chr(letter)])        
        return text_string
    
    def build_index(self):        
        ''' Override order of ASCII sets (32 to 125)'''
        self.ind = []
        chars = []
        for i in range(0,self.size-1):
            j = 32+(i*(94//self.size))
            k = 32+((i+1)*(94//self.size))
            chars.extend([[j,k]])
        chars.extend([[(94//self.size)*(self.size-1)+32, 127]])
        for i in range(self.size):
            for i in range(chars[self.plugs[i]][0],chars[self.plugs[i]][1]):
                self.ind.extend([i])

In [4]:
e = Enigma()
e.set_machine()
s = e.settings(True)
em = e.encrypt(msg)
print('\n', em[:200])

print('----------')
# #reset machine
er = Enigma()
er.set_machine([s[0], s[1], s[2]], [s[3], s[4], s[5]], s[6], s[7], s[8])
er.settings(False)

dm = e.decrypt(em)
print(dm[:200])


*** SETTINGS FOR MACHINE ***
R1: '
R2: q
R3: _

R1 Shift: 3
R2 Shift: 41
R3 Shift: 55

Plug Size: 8
Plug Order: [0, 6, 1, 3, 7, 5, 4, 2]

 5[[xLRi+/LAB!7Gb@v++bZ4^\\Dm6@: ?['AbRa]Rt5SBFIKsh&)X~i1DO8S97{@. ~[}OK.83Q.~$f?us'~33jshRqBrw:"&&gUTi78=/yF(Fc@hwwxe-&HH2`T|^]T'^R[WJ]/31Bm8%v Hj.6Yifnq'QDl:|L+U8AB@$no;CCc)K#jvyY~ZLG!4lU=~\r!&`Nih_2
----------
The first use of the term 'cryptograph' (as opposed to 'cryptogram') dates back to the 19th century-originating from 'The Gold-Bug,' a story by Edgar Allan Poe.[12][13][broken footnote] Until modern t


In [5]:
message_1 = "The main classical cipher types are transposition ciphers, which rearrange the order of letters in a message (e.g., 'hello world' becomes 'ehlol owrdl' in a trivially simple rearrangement scheme), and substitution ciphers, which systematically replace letters or groups of letters with other letters or groups of letters (e.g., 'fly at once' becomes 'gmz bu podf' by replacing each letter with the one following it in the Latin alphabet)"
message_2 = "In India, the 2000-year-old Kamasutra of Vatsyayana speaks of two different kinds of ciphers called Kautiliyam and Mulavediya. In the Kautiliyam, the cipher letter substitutions are based on phonetic relations, such as vowels becoming consonants. In the Mulavediya, the cipher alphabet consists of pairing letters and using the reciprocal ones"
message_3 = "David Kahn notes in The Codebreakers that modern cryptology originated among the Arabs, the first people to systematically document cryptanalytic methods.[24] Al-Khalil (717–786) wrote the Book of Cryptographic Messages, which contains the first use of permutations and combinations to list all possible Arabic words with and without vowels"
message_4 = "Ciphertexts produced by a classical cipher (and some modern ciphers) will reveal statistical information about the plaintext, and that information can often be used to break the cipher. After the discovery of frequency analysis, perhaps by the Arab mathematician and polymath Al-Kindi (also known as Alkindus) in the 9th century,[26] nearly all such ciphers could be broken by an informed attacker. Such classical ciphers still enjoy popularity today, though mostly as puzzles (see cryptogram). Al-Kindi wrote a book on cryptography entitled Risalah fi Istikhraj al-Mu'amma (Manuscript for the Deciphering Cryptographic Messages), which described the first known use of frequency analysis cryptanalysis techniques.[26][27]"
message_5 = "Nonetheless, good modern ciphers have stayed ahead of cryptanalysis; it is typically the case that use of a quality cipher is very efficient (i.e., fast and requiring few resources, such as memory or CPU capability), while breaking it requires an effort many orders of magnitude larger, and vastly larger than that required for any classical cipher, making cryptanalysis so inefficient and impractical as to be effectively impossible."

In [6]:
message_list = [ message_1, message_2, message_3, message_4, message_5]

In [7]:
def code_message(message):
    em = Enigma()
    em.set_machine() #Randomise
    em.build_index()
    me = em.encrypt(message)
    return [em.settings(), me]
def decode_message(s, message):
    d = Enigma()
    d.set_machine([s[0], s[1], s[2]], [s[3], s[4], s[5]], s[6], s[7], s[8])
    d_settings = d.settings()
    message_d = d.decrypt(message)
    return message_d

In [8]:
encoded_list = []
for message in message_list:
    c = code_message(message)
    encoded_list.extend([c])

for msg in encoded_list:
    print(msg, '\n')

[['S', 'k', 'g', 3, 44, 80, 18, [7, 15, 6, 12, 3, 17, 0, 13, 14, 11, 2, 16, 8, 1, 10, 4, 5, 9], 25], 'd]]>40|Mu E$HK+Z[2*H)=VVkuAS\\E3Qm|^?deuN[[lsX%,02Z+\';TTFmG4RFmG@F}_^cfvO#$}mXXf1UrI;saV\'g;\\_D1vD^R>_E:{Z+!fga5*(sV4qlVVehZ6TrZH>cn^23=M ,|\'hZ\\00ZzX=($;)MZ7N=vw]O1^58\\3*dkq>XfR7.8;K$45h;[_;2kAA)`Y#`wP!Frwg&Rr5f;)#a%4=oQQ/30EO/a0y/X/,qe>g(p,-wz+HC4Gi9Mp=so| _$d_MXwbp>yJ(iut,{\'oF5m?5Sk3>D(!11d`<~,pdtKd(SYI9 [OF(W=6+Eux?|!?Yve~YX|\'i@#h*6+} \'TA)7=MpqlxF=/V>_wBw*mejVf[f[8L<")V<X;8DCkAG(`cfxju/esja\'--Yy{BtTeehZ)FqmQ\\~n36^j  yIaJc^h*35&J&Tk_'] 

[['O', 'P', 'v', 19, 21, 36, 31, [18, 0, 10, 22, 28, 21, 26, 8, 29, 14, 2, 20, 17, 6, 19, 13, 23, 7, 5, 11, 15, 27, 3, 24, 16, 30, 4, 25, 9, 1, 12], 25], "mz~ITf(E,9g'L BEaS-G;RG-Ue+~KSKRHu'?R~o+4AEtlrR?aTXMHp*0GH U<M=wU4+5f+;yLn'bG5n&@MVf9e(Wh*?J~cSJHLd90R#t#J(?ab4RFd9 isa\\;+5yS^MKn9g'L I0i=i@ukEm!4hge9~(Wh*?Mse'gcG l5)Ht#gi=iUuJ~ak;Mmal;+~oT4|goT;h5c9,csa'u{Fs!4J#c=4RH \\A-Lll3)LcUK(Fg9e{FsU1RFtlxMKn9i'L ~5HEv*f(?a!4hge9e(Wh*eMEl2tRme'4*

In [9]:
decoded_list =[]
for message in encoded_list:
    decoded_list.extend([decode_message(message[0], message[1])])

for msg in decoded_list:
    print(msg, '\n')

The main classical cipher types are transposition ciphers, which rearrange the order of letters in a message (e.g., 'hello world' becomes 'ehlol owrdl' in a trivially simple rearrangement scheme), and substitution ciphers, which systematically replace letters or groups of letters with other letters or groups of letters (e.g., 'fly at once' becomes 'gmz bu podf' by replacing each letter with the one following it in the Latin alphabet) 

In India, the 2000-year-old Kamasutra of Vatsyayana speaks of two different kinds of ciphers called Kautiliyam and Mulavediya. In the Kautiliyam, the cipher letter substitutions are based on phonetic relations, such as vowels becoming consonants. In the Mulavediya, the cipher alphabet consists of pairing letters and using the reciprocal ones 

David Kahn notes in The Codebreakers that modern cryptology originated among the Arabs, the first people to systematically document cryptanalytic methods.[24] Al-Khalil (717?786) wrote the Book of Cryptographic Mes